In [1]:
!pip3 install numpy
!pip3 install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095426 sha256=da042c53c4befa343434a3f5bce58f5e31a150631e589f81e7465591aed69d42
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [2]:
import pandas as pd
import numpy as np
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV

In [3]:
df = pd.read_csv ("https://raw.githubusercontent.com/singhsidhukuldeep/Recommendation-System/master/data/ratings.csv")

In [15]:
df.groupby('movieId').mean()

,userId,rating
movieId,,
1,306.530233,3.920930
2,329.554545,3.431818
3,283.596154,3.259615
4,219.857143,2.357143
5,299.571429,3.071429
...,...,...
193581,184.000000,4.000000
193583,184.000000,3.500000
193585,184.000000,3.500000


In [11]:
df.drop(['timestamp'], axis=1, inplace=True)
df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [46]:
min_ratings = 100
filter_items = df['movieId'].value_counts() > min_ratings
filter_items = filter_items[filter_items].index.tolist()

df_new = df[df['movieId'].isin(filter_items)]

min_user_ratings = 10
filter_users = df_new['userId'].value_counts() > min_user_ratings
filter_users = filter_users[filter_users].index.tolist()

df_new = df_new[df_new['userId'].isin(filter_users)]
print('The original data frame shape:\t{}'.format(df.shape))
print('The new data frame shape:\t{}'.format(df_new.shape))

The original data frame shape:	(100836, 3)
The new data frame shape:	(19130, 3)


In [55]:
df_new.groupby('userId').count()

,movieId,rating
userId,,
1,56,56
4,34,34
5,26,26
6,49,49
7,46,46
...,...,...
606,92,92
607,50,50
608,119,119


In [49]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_new[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.25)

algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

predictions

[Prediction(uid=452, iid=1222, r_ui=5.0, est=4.854991038258408, details={'was_impossible': False}),
 Prediction(uid=522, iid=7361, r_ui=4.0, est=4.365843386889675, details={'was_impossible': False}),
 Prediction(uid=298, iid=231, r_ui=2.5, est=2.835409557994581, details={'was_impossible': False}),
 Prediction(uid=607, iid=318, r_ui=5.0, est=4.229049842398648, details={'was_impossible': False}),
 Prediction(uid=362, iid=3147, r_ui=4.0, est=4.431421843012678, details={'was_impossible': False}),
 Prediction(uid=599, iid=2, r_ui=2.5, est=3.163517461795948, details={'was_impossible': False}),
 Prediction(uid=229, iid=329, r_ui=4.0, est=3.4274168162852767, details={'was_impossible': False}),
 Prediction(uid=40, iid=367, r_ui=4.0, est=3.581529338263306, details={'was_impossible': False}),
 Prediction(uid=437, iid=110, r_ui=4.0, est=3.9604441976606504, details={'was_impossible': False}),
 Prediction(uid=181, iid=349, r_ui=4.0, est=3.1265126825241443, details={'was_impossible': False}),
 Predic

In [50]:
# Then compute RMSE
evaluation = cross_validate(algo, data, measures=['RMSE','MAE'], cv= 5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8166  0.8161  0.8140  0.8110  0.8152  0.8146  0.0020  
MAE (testset)     0.6227  0.6261  0.6229  0.6157  0.6197  0.6214  0.0035  
Fit time          0.23    0.35    0.34    0.36    0.34    0.32    0.05    
Test time         0.03    0.05    0.21    0.04    0.04    0.08    0.07    


In [52]:
unique_ids = df_new['movieId'].unique()
len(unique_ids)

134

In [58]:
# get the list of the ids that the userid 13618 has watched
iids606 = df_new.loc[df_new['userId']==606, 'movieId']
# remove the rated movies for the recommendations
movies_to_predict = np.setdiff1d(unique_ids,iids606)
movies_to_predict

array([    2,     6,    10,    34,    39,   150,   153,   161,   165,
         185,   208,   253,   292,   316,   329,   344,   349,   364,
         377,   380,   434,   457,   539,   586,   588,   595,   608,
         733,   736,  1036,  1073,  1197,  1213,  1265,  2683,  2918,
        3793,  4878, 33794, 58559, 68954, 79132])

In [59]:
#use SVD() model to predict the top 10 movies for userId 606.
my_recs = []
for iid in movies_to_predict:
 my_recs.append((iid, algo.predict(uid='606',iid=iid).est))
pd.DataFrame(my_recs, columns=['iid', 'predictions']).sort_values('predictions', ascending=False).head(10)

,iid,predictions
31,1197,4.319423
32,1213,4.269550
26,608,4.156907
40,68954,4.134310
39,58559,4.132225
37,4878,4.070089
21,457,4.034487
41,79132,4.032562
33,1265,3.965181
17,364,3.947337
